# 8강) Dense Embedding 을 활용한 ODQA 시스템 만들기

### Requirements

In [1]:
%%bash
# install packages
pip install tqdm==4.64.1 -q
pip install datasets==2.12.0 -q
pip install transformers==4.24.0 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.3 MB/s eta 0:00:00


In [1]:
import torch
import random
from pprint import pprint

## Dense Embedding 을 활용한 Open-domain Question Answering 시스템 만들기


5강에서 배운대로 dense embedding을 만드는 encoder 을 학습시키기

또는 학습된 encoder 파일을 가져와서 진행하기

In [22]:
# google drive 에 올려둔 미리 학습해둔 인코더 불러오기
from transformers import BertModel, BertPreTrainedModel, BertConfig, AutoTokenizer

class BertEncoder(BertPreTrainedModel):
  def __init__(self, config):
    super(BertEncoder, self).__init__(config)

    self.bert = BertModel(config)
    self.init_weights()
      
  def forward(self, input_ids, 
              attention_mask=None, token_type_ids=None):
  
      outputs = self.bert(input_ids,
                          attention_mask=attention_mask,
                          token_type_ids=token_type_ids)
      
      pooled_output = outputs[1]

      return pooled_output

model_checkpoint = 'klue/bert-base'
p_encoder = BertEncoder.from_pretrained(model_checkpoint).to("cuda")
q_encoder = BertEncoder.from_pretrained(model_checkpoint).to("cuda")

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.de

## Dense Embedding을 활용하여 passage retrieval 실습해보기

In [3]:
from datasets import load_dataset
dataset = load_dataset("squad_kor_v1")

  0%|          | 0/2 [00:00<?, ?it/s]

사전에 학습한 passage encoder, question encoder을 이용해 dense embedding 생성

생성된 embedding에 dot product를 수행 => Document들의 similarity ranking을 구함

Top-5개의 passage를 retrieve 하고 ground truth와 비교하기

In [4]:
def to_cuda(batch):
  return tuple(t.cuda() for t in batch)

In [24]:
random.seed(2023)
valid_corpus = list(set([example['context'] for example in dataset['validation']]))[:10]
sample_idx = random.choice(range(len(dataset['validation'])))
query = dataset['validation'][sample_idx]['question']
ground_truth = dataset['validation'][sample_idx]['context']

if not ground_truth in valid_corpus:
  valid_corpus.append(ground_truth)

In [6]:
def get_relevant_doc(q_encoder, p_encoder, query, k=1):
    with torch.no_grad():
        p_encoder.eval()
        q_encoder.eval()

        q_seqs_val = tokenizer([query], padding="max_length", truncation=True, return_tensors='pt').to('cuda')
        q_emb = q_encoder(**q_seqs_val).to('cpu')  #(num_query, emb_dim)

        p_embs = []
        for p in valid_corpus:
            p = tokenizer(p, padding="max_length", truncation=True, return_tensors='pt').to('cuda')
            p_emb = p_encoder(**p).to('cpu').numpy()
            p_embs.append(p_emb)

    p_embs = torch.Tensor(p_embs).squeeze()  # (num_passage, emb_dim)
    dot_prod_scores = torch.matmul(q_emb, torch.transpose(p_embs, 0, 1))

    rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()
    
    return dot_prod_scores.squeeze(), rank[:k]

In [7]:
print("{} {} {}".format('*'*20, 'Ground Truth','*'*20))
print("[Search query]\n", query, "\n")
pprint(ground_truth, compact=True)

******************** Ground Truth ********************
[Search query]
 불법행위 당시에는 예견할 수 없었던 새로운 손해 또는 확대된 손해의 예는? 

('민법 766조 1항 소정의 손해 및 가해자를 안다고 하는 경우에 손해발생 사실을 안다는 것은 단순히 손해발생의 사실만을 안 때라는 뜻이 '
 '아니고 가해행위가 불법행위로써 이를 원인으로 하여 손해배상을 소구할 수 있다는 사실을 안 때 이자, 가해행위와 손해의 발생 사이에 '
 '인과관계가 있으며 위법하고 과실이 있는 것까지도 안 때이다. 가해행위와 이로 인한 현실적인 손해의 발생 사이에 시간적 간격이 있는 '
 '불법행위에 기한 손해배상채권의 경우, 소멸시효의 기산점이 되는 “불법행위를 한 날"의 의미는 단지 관념적이고 부동적인 상태에서 '
 '잠재적으로만 존재하고 있는 손해가 그 후 현실화되었다고 볼 수 있는 때, 다시 말하자면 손해의 결과발생이 현실적인 것으로 되었다고 할 수 '
 '있을 때 이고, 후유증 등으로 인하여 불법행위 당시에는 예견할 수 없었던 새로운 손해가 발생하였다거나 예상외로 손해가 확대된 경우에 '
 '있어서는 그러한 사유가 판명되었을 때 비로소 새로이 발생 또는 확대된 손해를 알았다고 보아야 한다. 다만, 현실화된 손해의 정도나 '
 '액수까지 구체적으로 알아야 하는 것은 아니다.')


In [8]:
_, doc_id = get_relevant_doc(q_encoder, p_encoder, query, k=1)

""" 상위 1개 문서를 추출했을 때 결과 확인 """
print("{} {} {}".format('*'*20, 'Result','*'*20))
print("[Search query]\n", query, "\n")
print(f"[Relevant Doc ID(Top 1 passage)]: {doc_id}")
pprint(valid_corpus[doc_id.item()])

******************** Result ********************
[Search query]
 불법행위 당시에는 예견할 수 없었던 새로운 손해 또는 확대된 손해의 예는? 

[Relevant Doc ID(Top 1 passage)]: tensor([8])
('미국 독립 혁명의 여파로 영국 정부는 영국계 인구가 큰 잔여 식민지들에서 정치적 그리고 사회적 불안에 민감하였다. 1837년에 루이-조셉 '
 '파피노가 이끌었던 로어 캐나다 반란과 1837년에서 1838년까지 윌리엄 라이언 매켄지가 이끌었던 어퍼 캐나다 반란을 겪고 난 후, 더럼 '
 '경이 영국령 북아메리카의 총독으로 임명한 다음부터 그는 이러한 국정의 문제점들을 연구하며 이런 불길한 사태들을 어떡해 해소하는데 힘썼다. '
 '그의 보고서에 토대로 그가 추천하였던 해결책은 어느정도 발전되었던 식민지에게 책임정부를 수행하는 권리를 승인하는 방법이였다. 그 때 당시 '
 '책임정부라는 정치적인 용어는 구체적으로 영국의 군주가 임명한 총독이 선출된 의원들로 구성된 식민지 의회의 뜻을 받아주는 정책을 뜻하였다.')


<ipython-input-6-a94f8a30a3d4>:15: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  p_embs = torch.Tensor(p_embs).squeeze()  # (num_passage, emb_dim)


In [9]:
""" 상위 5개를 추출하여 점수 확인 """
dot_prod_scores, rank = get_relevant_doc(q_encoder, p_encoder, query, k=5)

for i in range(5):
    print(rank[i])
    print("Top-%d passage with score %.4f" % (i+1, dot_prod_scores.squeeze()[rank[i]]))
    pprint(valid_corpus[rank[i]])

tensor(8)
Top-1 passage with score 89.6241
('미국 독립 혁명의 여파로 영국 정부는 영국계 인구가 큰 잔여 식민지들에서 정치적 그리고 사회적 불안에 민감하였다. 1837년에 루이-조셉 '
 '파피노가 이끌었던 로어 캐나다 반란과 1837년에서 1838년까지 윌리엄 라이언 매켄지가 이끌었던 어퍼 캐나다 반란을 겪고 난 후, 더럼 '
 '경이 영국령 북아메리카의 총독으로 임명한 다음부터 그는 이러한 국정의 문제점들을 연구하며 이런 불길한 사태들을 어떡해 해소하는데 힘썼다. '
 '그의 보고서에 토대로 그가 추천하였던 해결책은 어느정도 발전되었던 식민지에게 책임정부를 수행하는 권리를 승인하는 방법이였다. 그 때 당시 '
 '책임정부라는 정치적인 용어는 구체적으로 영국의 군주가 임명한 총독이 선출된 의원들로 구성된 식민지 의회의 뜻을 받아주는 정책을 뜻하였다.')
tensor(3)
Top-2 passage with score 82.2686
('믿음은 불도의 근본이며 복덕의 모태로써 궁극적인 깨달음에 이르기까지 인도자가 된다는 것으로 信心이 얼마나 중요한 것인지를 설명한 것이다. '
 '믿음은 迷信이나 盲信이 아닌 맑고 깨끗한 淨信이어야 한다고 賢首菩薩은 주장하고 있다. 청정한 믿음에서 모든 공덕이 생기기 때문이다. '
 '믿음을 실현하기 위해서는 청정한 계율을 지키고 바른 법을 따르는 것이다. 그렇게 하면 일체의 모든 것에 대해 집착이 없어져서 완전히 '
 '청정해지며, 그로 인해 위없는 마음을 얻을 수 있게 된다는 것이다. 이렇게 중요한 믿음에 대하여 『華嚴經』에는 十信의 명칭이나 십신보살의 '
 '실천행에 대해서는 설해지고 있지 않다. 그 이유를 法藏은 十住 내지 十地의 실천행에는 階位가 있을 수 있지만, 믿음에는 계위가 없기 '
 '때문에 나열하지 않는다고 설명하고 있다. 『大正藏』9, p.433, “信爲道元功德母 增長一切諸善法 除滅一切諸疑惑 示現開發無上道” '
 '『大正藏』9, p.433. 『探玄記』권4, (『大正藏』35, p

## 훈련된 MRC 모델 가져오기


In [10]:
import torch
from transformers import (
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer
)

In [11]:
model_name = 'sangrimlee/bert-base-multilingual-cased-korquad'
mrc_model = AutoModelForQuestionAnswering.from_pretrained(model_name).cuda()
mrc_model = mrc_model.eval()

In [21]:
qa_tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=False
)

In [13]:
def get_answer_from_context(context, question, model, tokenizer):
    encoded_dict = tokenizer.encode_plus(  
        question,
        context,
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    non_padded_ids = encoded_dict["input_ids"][: encoded_dict["input_ids"].index(tokenizer.pad_token_id)]
    full_text = tokenizer.decode(non_padded_ids)

    inputs = {
    'input_ids': torch.tensor([encoded_dict['input_ids']], dtype=torch.long),
    'attention_mask': torch.tensor([encoded_dict['attention_mask']], dtype=torch.long),
    'token_type_ids': torch.tensor([encoded_dict['token_type_ids']], dtype=torch.long)
    }

    inputs = {k: v.cuda() for k, v in inputs.items()}

    outputs = model(**inputs)
    start, end = torch.max(outputs.start_logits, axis=1).indices.item(), torch.max(outputs.end_logits, axis=1).indices.item()
    if start == 0 and end == 0:
        answer = "This is not answerable"
    else:
        answer = tokenizer.decode(encoded_dict['input_ids'][start:end+1])
    return answer

In [25]:
context = valid_corpus[doc_id.item()]
answer = get_answer_from_context(context, query, mrc_model, qa_tokenizer)
print("{} {} {}".format('*'*20, 'Result','*'*20))
print("[Search query]\n", query, "\n")
print(f"[Relevant Doc ID(Top 1 passage)]: {doc_id.item()}")
pprint(valid_corpus[doc_id.item()], compact=True)
print(f"[Answer Prediction from the model]: {answer}")

******************** Result ********************
[Search query]
 불법행위 당시에는 예견할 수 없었던 새로운 손해 또는 확대된 손해의 예는? 

[Relevant Doc ID(Top 1 passage)]: 8
('미국 독립 혁명의 여파로 영국 정부는 영국계 인구가 큰 잔여 식민지들에서 정치적 그리고 사회적 불안에 민감하였다. 1837년에 루이-조셉 '
 '파피노가 이끌었던 로어 캐나다 반란과 1837년에서 1838년까지 윌리엄 라이언 매켄지가 이끌었던 어퍼 캐나다 반란을 겪고 난 후, 더럼 '
 '경이 영국령 북아메리카의 총독으로 임명한 다음부터 그는 이러한 국정의 문제점들을 연구하며 이런 불길한 사태들을 어떡해 해소하는데 힘썼다. '
 '그의 보고서에 토대로 그가 추천하였던 해결책은 어느정도 발전되었던 식민지에게 책임정부를 수행하는 권리를 승인하는 방법이였다. 그 때 당시 '
 '책임정부라는 정치적인 용어는 구체적으로 영국의 군주가 임명한 총독이 선출된 의원들로 구성된 식민지 의회의 뜻을 받아주는 정책을 뜻하였다.')
[Answer Prediction from the model]: This is not answerable


## 통합해서 ODQA 시스템 구축

In [26]:
def open_domain_qa(query, corpus, p_encoder, q_encoder, mrc_model, tokenizer, qa_tokenizer, k=1):
    # 1. Retrieve k relevant docs by usign sparse matrix
    _, doc_id = get_relevant_doc(p_encoder, q_encoder, query, k=1)
    context = corpus[doc_id.item()]

    # 2. Predict answer from given doc by using MRC model
    answer = get_answer_from_context(context, query, mrc_model, qa_tokenizer)
    print("{} {} {}".format('*'*20, 'Result','*'*20))
    print("[Search query]\n", query, "\n")
    print(f"[Relevant Doc ID(Top 1 passage)]: {doc_id.item()}")
    pprint(corpus[doc_id.item()], compact=True)
    print(f"[Answer Prediction from the model]: {answer}")

In [29]:
query = input("Enter any question: ") # "대한민국의 대통령은 누구인가?"
open_domain_qa(query=query,
               corpus=valid_corpus,
               p_encoder=p_encoder,
               q_encoder=q_encoder,
               mrc_model=mrc_model,
               tokenizer=tokenizer,
               qa_tokenizer=qa_tokenizer,
               k=1)

Enter any question: 대한민국의 대통령은 누구인가?
******************** Result ********************
[Search query]
 대한민국의 대통령은 누구인가? 

[Relevant Doc ID(Top 1 passage)]: 8
('미국 독립 혁명의 여파로 영국 정부는 영국계 인구가 큰 잔여 식민지들에서 정치적 그리고 사회적 불안에 민감하였다. 1837년에 루이-조셉 '
 '파피노가 이끌었던 로어 캐나다 반란과 1837년에서 1838년까지 윌리엄 라이언 매켄지가 이끌었던 어퍼 캐나다 반란을 겪고 난 후, 더럼 '
 '경이 영국령 북아메리카의 총독으로 임명한 다음부터 그는 이러한 국정의 문제점들을 연구하며 이런 불길한 사태들을 어떡해 해소하는데 힘썼다. '
 '그의 보고서에 토대로 그가 추천하였던 해결책은 어느정도 발전되었던 식민지에게 책임정부를 수행하는 권리를 승인하는 방법이였다. 그 때 당시 '
 '책임정부라는 정치적인 용어는 구체적으로 영국의 군주가 임명한 총독이 선출된 의원들로 구성된 식민지 의회의 뜻을 받아주는 정책을 뜻하였다.')
[Answer Prediction from the model]: [CLS] 대한민국의 대통령은 누구인가? [SEP] 미국 독립 혁명의 여파로 영국 정부는 영국계 인구가 큰 잔여 식민지들에서 정치적 그리고 사회적 불안에 민감하였다. 1837년에 루이 - 조셉 파피노가 이끌었던 로어 캐나다 반란과 1837년에서 1838년까지 윌리엄 라이언 매켄지가 이끌었던 어퍼 캐나다 반란을 겪고 난 후, 더럼 경


### **콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다. 모델 라이선스 : MIT License

